In [98]:
#Import your dependencies
import platform
# from hdbcli import dbapi
from hana_ml.dataframe import ConnectionContext as cc
import requests
import json
import pandas as pd
from hana_ml import dataframe

#verify the architecture of Python
print ("Platform architecture: " + platform.architecture()[0])


Platform architecture: 64bit


In [105]:
#Initialize your connection
sap_hana_config_file = "env_cloud.json"
with open(sap_hana_config_file) as f:
    sap_hana_config = json.load(f)
    db_url  = sap_hana_config['url']
    db_port = sap_hana_config['port']
    db_user = sap_hana_config['user']
    db_pwd  = sap_hana_config['pwd']

    # Saurav: this took a while to debug. sslValidateCertificate must be set to false when connecting to an SAP HANA Cloud
conn = cc(db_url, db_port, db_user, db_pwd, sslValidateCertificate=False 
)
#If no errors, print connected
print('connected')


connected


In [102]:

api_url = "https://creativecommons.tankerkoenig.de/api/v4/stats?apikey=cffa4fb8-7a16-cd85-7946-263722530f15"
headers =  {"Content-Type":"application/json", "charset":"utf-8"}
response = requests.get(api_url, headers=headers)
json_data = response.json()



timestamp = json_data['timestamp']
data = [
    {'FuelType':fuel_type, 'Count': fuel_data['count'], 'Mean': fuel_data['mean'], 'Median': fuel_data['median'], 'Timestamp': timestamp}
    for fuel_type, fuel_data in json_data.items()
    if fuel_type not in ['timestamp', 'license', 'apiVersion']
    

]

    
       

df = pd.DataFrame(data)



print(df)


hana_df = dataframe.create_dataframe_from_pandas(conn, df, 'TANK_STATS',force=True,replace=True,drop_exist_tab=False)








  FuelType  Count   Mean  Median                 Timestamp
0       E5  11736  1.813   1.799  2023-07-12T21:24:57+0200
1      E10  11237  1.755   1.739  2023-07-12T21:24:57+0200
2   Diesel  12044  1.593   1.579  2023-07-12T21:24:57+0200


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.36it/s]
